In [ ]:
Deploy the Iris model you created in the previous module using Kubernetes.

In [1]:
# Install necessary libraries
!pip install pandas scikit-learn joblib flask flask-cors kubernetes requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 14.8 MB/s eta 0:00:00


In [3]:
# Install Kubernetes tools (if not installed)
!brew install minikube kubectl


==> Downloading https://formulae.brew.sh/api/formula.jws.json
==> Downloading https://formulae.brew.sh/api/cask.jws.json
To reinstall 1.35.0, run:
  brew reinstall minikube
To reinstall 1.32.2, run:
  brew reinstall kubernetes-cli


In [5]:
# Start Minikube
!minikube start --driver=docker


😄  minikube v1.35.0 on Darwin 15.3.1 (arm64)
✨  Using the docker driver based on existing profile
👍  Starting "minikube" primary control-plane node in "minikube" cluster
🚜  Pulling base image v0.0.46 ...
🏃  Updating the running docker "minikube" container ...
🐳  Preparing Kubernetes v1.32.0 on Docker 27.4.1 ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


In [33]:
# Verify Kubernetes is running
!kubectl get nodes
import pandas as pd
import urllib.request
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import joblib

# Download and load dataset
download_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
file_name = "iris.data"
urllib.request.urlretrieve(download_url, file_name)

df = pd.read_csv(file_name, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])

# Encode species labels
le = LabelEncoder()
df['species'] = le.fit_transform(df['species'])

# Split data
X = df.drop('species', axis=1)
y = df['species']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save model
joblib.dump(model, 'iris_model.pkl')


NAME       STATUS   ROLES           AGE     VERSION
minikube   Ready    control-plane   2d12h   v1.32.0


['iris_model.pkl']

In [53]:
le.__dict__

{'classes_': array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)}

In [57]:
# Create Flask API
flask_code = """from flask import Flask, send_from_directory, jsonify, request
import os
from flask_cors import CORS
import pickle as pkl
import joblib

app = Flask(__name__, static_folder="frontend", static_url_path="")
CORS(app)

label_dict = {0: "Iris-setosa", 1: 'Iris-versicolor', 2: 'Iris-virginica'}


model = joblib.load("iris_model.pkl") 
    

#  Serve index.html at root URL
@app.route('/')
def serve_index():
    return send_from_directory("frontend", "index.html")

#  Serve static files (if you have any CSS/JS)
@app.route('/<path:path>')
def serve_static_files(path):
    return send_from_directory("frontend", path)

#  Prediction API Endpoint
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    print(model.__dir__)
    predictions = label_dict[model.predict([data['features']])[0]]
    return jsonify({"message": "Prediction endpoint is working", "received_data": data, "prediction": predictions})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
"""

with open("app.py", "w") as f:
    f.write(flask_code)


In [41]:
import joblib
model = joblib.load("iris_model.pkl")
 

In [43]:
model

RandomForestClassifier(random_state=42)

In [5]:
# Test Flask API locally
!python app.py


 * Serving Flask app 'app'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.42:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Mar/2025 09:10:09] "GET / HTTP/1.1" 304 -
^C


In [7]:
dockerfile_backend = """FROM python:3.8
WORKDIR /app
COPY iris_model.pkl app.py /app/
RUN pip install flask flask-cors joblib numpy scikit-learn
CMD ["python", "app.py"]
"""

with open("Dockerfile", "w") as f:
    f.write(dockerfile_backend)


In [9]:
dockerfile_backend = """FROM python:3.8
WORKDIR /app
COPY iris_model.pkl app.py /app/
RUN pip install flask flask-cors joblib numpy scikit-learn
CMD ["python", "app.py"]
"""

with open("Dockerfile", "w") as f:
    f.write(dockerfile_backend)


In [3]:
# Build and run Docker container
!docker build -t iris-api .
!docker run -p 5000:5000 iris-api


[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 185B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.8              0.2s
[+] Building 0.3s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 185B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.8              0.3s
[+] Building 0.5s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 185B                                      

In [7]:
# Create backend deployment YAML
deployment_yaml = """apiVersion: apps/v1
kind: Deployment
metadata:
  name: iris-api
spec:
  replicas: 1
  selector:
    matchLabels:
      app: iris-api
  template:
    metadata:
      labels:
        app: iris-api
    spec:
      containers:
      - name: iris-api
        image: iris-api:latest
        ports:
        - containerPort: 5000
"""

with open("backend-deployment.yaml", "w") as f:
    f.write(deployment_yaml)


In [9]:
# Apply backend deployment
!kubectl apply -f backend-deployment.yaml


deployment.apps/iris-api created


In [11]:
# Create backend service YAML
service_yaml = """apiVersion: v1
kind: Service
metadata:
  name: iris-service
spec:
  selector:
    app: iris-api
  ports:
    - protocol: TCP
      port: 5000
      targetPort: 5000
  type: LoadBalancer
"""

with open("backend-service.yaml", "w") as f:
    f.write(service_yaml)


In [13]:
# Apply backend service
!kubectl apply -f backend-service.yaml


service/iris-service configured


In [80]:
html_code = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Iris Flower Classifier</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            margin: 40px;
        }
        input {
            display: block;
            margin: 10px auto;
            padding: 8px;
        }
        button {
            background-color: green;
            color: white;
            padding: 10px;
            border: none;
            cursor: pointer;
        }
        button:hover {
            background-color: darkgreen;
        }
    </style>
</head>
<body>

    <h1>Iris Flower Classifier</h1>
    
    <label>Sepal Length: <input type="number" id="sepal-length" step="0.1"></label>
    <label>Sepal Width: <input type="number" id="sepal-width" step="0.1"></label>
    <label>Petal Length: <input type="number" id="petal-length" step="0.1"></label>
    <label>Petal Width: <input type="number" id="petal-width" step="0.1"></label>

    <button id="predict-button">Predict</button>
    
    <h3 id="prediction-result"></h3>

    <script>
        const API_URL = "https://iris-ml-app.herokuapp.com/predict"; // Change to your API URL

        document.getElementById("predict-button").addEventListener("click", async function() {
            const data = {
                sepal_length: parseFloat(document.getElementById("sepal-length").value),
                sepal_width: parseFloat(document.getElementById("sepal-width").value),
                petal_length: parseFloat(document.getElementById("petal-length").value),
                petal_width: parseFloat(document.getElementById("petal-width").value)
            };

            // Ensure all inputs are filled
            if (Object.values(data).some(isNaN)) {
                document.getElementById("prediction-result").innerText = "Please enter valid values.";
                return;
            }

            try {
                const response = await fetch(API_URL, {
                    method: "POST",
                    headers: {
                        "Content-Type": "application/json"
                    },
                    body: JSON.stringify(data)
                });

                if (!response.ok) {
                    throw new Error(`Server error: ${response.status}`);
                }

                const result = await response.json();
                document.getElementById("prediction-result").innerText = "Prediction: " + result.prediction;

            } catch (error) {
                document.getElementById("prediction-result").innerText = "Error making prediction.";
                console.error("Prediction Error:", error);
            }
        });
    </script>

</body>
</html>
"""

with open("index.html", "w") as f:
    f.write(html_code)


In [21]:
frontend_yaml = """apiVersion: apps/v1
kind: Deployment
metadata:
  name: iris-frontend
spec:
  replicas: 1
  selector:
    matchLabels:
      app: iris-frontend
  template:
    metadata:
      labels:
        app: iris-frontend
    spec:
      containers:
      - name: iris-frontend
        image: nginx:alpine
        ports:
        - containerPort: 80
        volumeMounts:
        - name: html-volume
          mountPath: /usr/share/nginx/html
      volumes:
      - name: html-volume
        hostPath:
          path: /path/to/index.html
"""

with open("frontend-deployment.yaml", "w") as f:
    f.write(frontend_yaml)


In [98]:
frontend_service_yaml = """apiVersion: v1
kind: Service
metadata:
  name: frontend-service
spec:
  selector:
    app: iris-frontend
  ports:
    - protocol: TCP
      port: 80
      targetPort: 80
  type: LoadBalancer
serviceaccount:
  name: dashboard-admin-sa
  namespace: kube-system
"""

with open("frontend-service.yaml", "w") as f:
    f.write(frontend_service_yaml)


In [31]:
!kubectl apply -f frontend-service.yaml


service/frontend-service created


In [23]:
!kubectl apply -f frontend-deployment.yaml


deployment.apps/iris-frontend created


In [33]:
!kubectl get services


NAME               TYPE           CLUSTER-IP       EXTERNAL-IP   PORT(S)          AGE
frontend-service   LoadBalancer   10.105.192.56    <pending>     80:31613/TCP     16s
iris-model         NodePort       10.99.232.25     <none>        5000:32616/TCP   90m
iris-service       LoadBalancer   10.111.206.231   <pending>     5000:30811/TCP   90m
kubernetes         ClusterIP      10.96.0.1        <none>        443/TCP          101m


In [11]:
!minikube service frontend-service --url


http://127.0.0.1:51830
❗  Because you are using a Docker driver on darwin, the terminal needs to be open to run it.
^C


In [41]:
!git init

Reinitialized existing Git repository in /Users/aquilasaguiya/ANA680/M4/.git/
fatal: pathspec 'README.md' did not match any files
On branch main

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	Dockerfile
	app.py
	backend-deployment.yaml
	backend-service.yaml
	frontend-deployment.yaml
	frontend-service.yaml
	index.html
	iris.data
	iris_model.pkl
	iris_project.ipynb

nothing added to commit but untracked files present (use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/aquilas23/iris-project-680.git'


In [47]:
!git add .


In [49]:
!git commit -m "Initial commit: Iris ML Model with Kubernetes"


[main (root-commit) c0cb156] Initial commit: Iris ML Model with Kubernetes
 11 files changed, 16419 insertions(+)
 create mode 100644 .ipynb_checkpoints/iris_project-checkpoint.ipynb
 create mode 100644 Dockerfile
 create mode 100644 app.py
 create mode 100644 backend-deployment.yaml
 create mode 100644 backend-service.yaml
 create mode 100644 frontend-deployment.yaml
 create mode 100644 frontend-service.yaml
 create mode 100644 index.html
 create mode 100644 iris.data
 create mode 100644 iris_model.pkl
 create mode 100644 iris_project.ipynb


In [51]:
!git branch -M main


In [53]:
!git push -u origin main


Enumerating objects: 14, done.
Counting objects: 100% (14/14), done.
Delta compression using up to 8 threads
Compressing objects: 100% (14/14), done.
Writing objects: 100% (14/14), 47.24 KiB | 3.94 MiB/s, done.
Total 14 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), done.
To https://github.com/aquilas23/iris-project-680.git
 * [new branch]      main -> main
branch 'main' set up to track 'origin/main'.


In [55]:
!brew tap heroku/brew && brew install heroku


==> Downloading https://formulae.brew.sh/api/formula.jws.json
==> Downloading https://formulae.brew.sh/api/cask.jws.json
heroku 10.1.0 is already installed but outdated (so it will be upgraded).
Error: Cannot install under Rosetta 2 in ARM default prefix (/opt/homebrew)!
To rerun under ARM use:
    arch -arm64 brew install ...
To install under x86_64, install Homebrew into /usr/local.


In [57]:
!brew tap heroku/brew && brew install heroku


heroku 10.1.0 is already installed but outdated (so it will be upgraded).
Error: Cannot install under Rosetta 2 in ARM default prefix (/opt/homebrew)!
To rerun under ARM use:
    arch -arm64 brew install ...
To install under x86_64, install Homebrew into /usr/local.
